In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import emoji

from glove_vec import read_glove_vectors

from keras.models import Sequential
from keras.layers import Input,SimpleRNN,LSTM,Dropout,Dense,Activation
from keras.utils import np_utils



Using TensorFlow backend.


In [2]:
train=pd.read_csv("train_emoji.csv",header=None)
test=pd.read_csv("test_emoji.csv",header=None)

In [4]:
emoji_dict={"0":":bat:","1":":kiss:","2":":heart:","3":":smile:","4":":baseball:"}

In [5]:
for ix in emoji_dict.keys():
    
    print(ix,emoji.emojize(emoji_dict[ix],use_aliases=True))

0 🦇
1 💋
2 ❤
3 😄
4 ⚾


In [6]:
X_train = train[0]
Y_train = train[1]

X_test = test[0]
Y_test = test[1]

In [7]:
for ix in range(X_train.shape[0]):
        X_train[ix]=X_train[ix].split()
        

for ix in range(X_test.shape[0]):
    X_test[ix]=X_test[ix].split()
    
Y_train=np_utils.to_categorical(Y_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
print(X_train[0],Y_train[0])

['never', 'talk', 'to', 'me', 'again'] [0. 0. 0. 1. 0.]


In [9]:
emb=read_glove_vectors("glove.6B.50d.txt")

In [30]:
embedding_train=np.zeros((X_train.shape[0],10,50))

embedding_test=np.zeros((X_test.shape[0],10,50))

for ix in range(X_train.shape[0]):
    
    for ij in range(len(X_train[ix])):
        
        embedding_train[ix][ij]=emb[X_train[ix][ij].lower()]
        

for ix in range(X_test.shape[0]):
    
    for ij in range(len(X_test[ix])):
        
        embedding_test[ix][ij]=emb[X_test[ix][ij].lower()]
        
        


In [31]:
print(embedding_train.shape)

(132, 10, 50)


In [35]:
model=Sequential()
model.add(LSTM(32,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(16,return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5,activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_17 (LSTM)               (None, 10, 32)            10624     
_________________________________________________________________
dropout_17 (Dropout)         (None, 10, 32)            0         
_________________________________________________________________
lstm_18 (LSTM)               (None, 16)                3136      
_________________________________________________________________
dropout_18 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 5)                 85        
Total params: 13,845
Trainable params: 13,845
Non-trainable params: 0
_________________________________________________________________


In [36]:
model.compile(loss="categorical_crossentropy", optimizer="RMSprop", metrics=['accuracy'])

In [37]:
hist = model.fit(embedding_train, Y_train, epochs=50, batch_size=12,shuffle=True, validation_split=0.05)

Train on 125 samples, validate on 7 samples
Epoch 1/50
125/125 [==============================] - 2s 15ms/step - loss: 1.5742 - acc: 0.2640 - val_loss: 1.7138 - val_acc: 0.1429
Epoch 2/50
125/125 [==============================] - 0s 2ms/step - loss: 1.5500 - acc: 0.3120 - val_loss: 1.7156 - val_acc: 0.1429
Epoch 3/50
125/125 [==============================] - 0s 2ms/step - loss: 1.5009 - acc: 0.3520 - val_loss: 1.7154 - val_acc: 0.1429
Epoch 4/50
125/125 [==============================] - 0s 2ms/step - loss: 1.4778 - acc: 0.3360 - val_loss: 1.6972 - val_acc: 0.2857
Epoch 5/50
125/125 [==============================] - 0s 2ms/step - loss: 1.4668 - acc: 0.3600 - val_loss: 1.6137 - val_acc: 0.2857
Epoch 6/50
125/125 [==============================] - 0s 2ms/step - loss: 1.4614 - acc: 0.3680 - val_loss: 1.5255 - val_acc: 0.4286
Epoch 7/50
125/125 [==============================] - 0s 2ms/step - loss: 1.3693 - acc: 0.5440 - val_loss: 1.5894 - val_acc: 0.2857
Epoch 8/50
125/125 [===========

In [38]:
pred=model.predict_classes(embedding_test)

In [39]:
acc=float(sum(pred==Y_test))/embedding_test.shape[0]

In [41]:
print(acc)

0.6964285714285714


In [56]:
def print_emoji(label):
    return emoji.emojize(emoji_dict[label], use_aliases=True)

In [57]:
for ix in range(embedding_test.shape[0]):
    
    if pred[ix] != Y_test[ix]:
        print (X_test[ix], print_emoji(Y_test[ix]), print_emoji(pred[ix]))

KeyError: 2